Author: Farid Javadnejad\
Institution: Leica Geosystems\
Latest Update: 1/22/2024

#Leica RC: QBR Reports

### Import Required Libraries

In [ ]:
# Import NumPy, Pandas, PyPlot SeaBorn libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import seaborn as sns

# Import Google Colab filer upload libs
import io
from google.colab import files
from google.colab import drive

# Import Google Sheet access authentication libs
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

#Files & folders
import shutil
import os

#time
from datetime import datetime
from pandas.tseries.offsets import MonthEnd

#Excel file handling
!pip install xlsxwriter
import xlsxwriter

# notifier
from IPython.display import Javascript


Global: Colors & Font Settings

In [ ]:
#Colors
global COLOR_SET_google
COLOR_SET_google = ['#F3C302', '#4486F4', '#51B457', '#E44B31', '#808080']

#Set default font size
global FONT_SIZE
FONT_SIZE = "13"


global PALLETE
PALLETE = ['viridis', 'magma', 'plasma', 'tab20', 'tab10', 'paired']


Global: Directory and Regions

In [ ]:
#Define global variables
global PROCESS_MONTH

global DIRECTORY


#create a list of all Regions
global ALL_REGIONS
ALL_REGIONS = ['LATAM', 'EM EMEA', 'India', 'Nordics', 'Russia', 'ANZ', 'China', 'Asia', 'UK/BX', 'Central EU','South EU','US/CA']

global ALL_PANREGIONS
ALL_PANREGIONS = {'LATAM': 'AMER',
    'EM EMEA': 'EU',
    'India': 'ANZ+India',
    'Nordics': 'EU',
    'ANZ': 'ANZ+India',
    'China': 'China',
    'Asia': 'Asia',
    'UK/BX': 'EU',
    'Central EU': 'EU',
    'South EU': 'EU',
    'US/CA': 'AMER',
    'Russia': 'Russia'}


## Custom Functions

###Color Functions

f: Color Picker Function

In [ ]:
def gradient_color_picker(i,n, pallete = 'PALLETE'):

  magma = ['#000004', '#120d31', '#331067', '#59157e', '#7e2482', '#a3307e', '#c83e73', '#e95462', '#fa7d5e',' #fea973', '#fed395', '#fcfdbf']

  tab20 = ['#1f77b4', '#17becf', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b',
           '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#6baed6', '#9edae5']

  tab10 = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']

  viridis = ['#440154', '#481b6d', '#46327e', '#3f4788', '#365c8d', '#2e6e8e', '#277f8e', '#21918c',
             '#1fa187', '#2db27d', '#4ac16d', '#73d056', '#a0da39', '#d0e11c', '#fde725']

  plasma = ['#0d0887', '#350498', '#5302a3', '#6f00a8', '#8b0aa5', '#a31e9a', '#b83289', '#cc4778', '#db5c68',
          '#e97158', '#f48849', '#fba238', '#febd2a', '#fada24', '#f0f921']

  paired =['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c',
           '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']


  if pallete == 'viridis':
    pallete = viridis
  elif pallete == 'plasma':
    pallete = plasma
  elif pallete == 'tab20':
    pallete = tab20
  elif pallete == 'tab10':
    pallete = tab10
  else:
    pallete = magma

  sizeOfList = len(pallete) - 1
  if n < 0 or n > sizeOfList:
    raise ValueError #The accepted values are between 0 - {sizeOfList}

  if i > n:
    raise ValueError #i can not be larget than {sizeOfList}

  idx = sizeOfList*i//n

  return pallete[idx]

###Reading Functions

Generate Keyword from List

In [ ]:
def keyword_from_list(li: list):
  ''' Gets a list of strings and returns one string as keyword'''

  str_list = filter(lambda item: item !='', li)  #remove empty items from list
  key_words ='|'.join(str_list)  #keyword Generator
  return key_words

Goolge Workbook to DataFrame

In [ ]:
def workbook_to_dataframe(work_book, work_sheet):
  '''Gets a GoogleSheet workbook and worksheet name oand retrun a dataframe'''

  ws = work_book.worksheet(work_sheet) #read the worksheet
  rows = ws.get_all_values() #get_all_values gives a list of rows
  df = pd.DataFrame.from_records(rows) #create a dataframe from all values
  df.replace('', np.nan) #data cleanup
  return df



f: Read GoogleSheet by URL

In [ ]:
def read_gs_by_url(url: str):
  ''' Read a workbook by url '''

  gc = gspread.authorize(creds)
  wb = gc.open_by_url(url)
  return wb

f: Cast df colum to a dictionry

In [ ]:
def column_to_dict(column, output = True):
  ''' Cast dataframe columnto a dictionry '''
  dct = {}
  i = 0
  for item in column:
    if item != '':
      dct[i]= item
      i +=1
  if output:
    for a,b in dct.items():
      print(a, ':', b)
  return dct


### Pre-processing Functions

f: Replace df header with 1st row

In [ ]:
def dataframe_header(df: pd.DataFrame):
   ''' Python Pandas Replacing Header with Top Row '''
   df_header = df.iloc[0] #grab the first row for the header
   df = df[1:] #take the data less the header row
   df.columns = df_header #set the header row as the df header
   return df

f: Get YYYY-MM from dataset

In [ ]:
def format_month(date_string):
    parts = date_string.split('.')
    month = str(int(parts[0])).zfill(2)
    year = parts[1]
    formatted_date = f"{year}-{month}"
    return formatted_date

f: Clean-up & Rename

In [ ]:
def clean_qty(df):
    if 'Qty' in df.columns:
        replace_list = [' PC.', ' AU', ',', 'WK', 'YR', '*']
        for label in replace_list:
            df['Qty'] = df['Qty'].replace(label, '')
        df['Qty'] = df['Qty'].replace('', np.nan).fillna(0)
    return df

def filter_regions(df):
    if 'Region' in df.columns:
        world_sales_filter = df['Region'].str.contains("World GEO").fillna(False)
        df = df.loc[~world_sales_filter.values]
    return df

def clean_region_names(df):
    if 'Region' in df.columns:
        df.loc[:, 'Region'] = df['Region'].str.replace('GSR', '')
        df.loc[:, 'Region'] = df['Region'].str.replace('US/CAN', 'US/CA')
        df.loc[:, 'Region'] = df['Region'].str.replace('Europe', 'EU')
        df.loc[:, 'Region'] = df['Region'].str.replace('Emerging', 'EM')
    return df

def convert_to_numeric(df):
    for column in ['Billing', 'Sales', 'Qty']:
        if column in df.columns:
            df[column] = df[column].astype(float)
    return df

def manage_dates(df):
    if 'MM_YY' in df.columns:
        df = df.copy()
        df = df[df['MM_YY'].astype(str).str.match(r'^\D')==False]
        df['MM_YY'] = df['MM_YY'].str.lstrip('0')
        df = df.dropna(subset=['MM_YY'])
        df['Date'] = pd.to_datetime(df['MM_YY'], format='%m.%Y')
        df['Date'] = df['Date'] + MonthEnd(1)
        cols = df.columns.tolist()
        cols.insert(0, cols.pop(cols.index('Date')))
        df = df[cols]
    return df

def drop_non_numeric_rows(df, column_name):
    # Copy the original DataFrame to avoid modifying it in place
    new_df = df.copy()

    # Use pd.to_numeric to convert the values in the specified column to numeric, setting errors='coerce' to convert non-numeric values to NaN
    new_df[column_name] = pd.to_numeric(new_df[column_name], errors='coerce')

    # Use boolean indexing to drop rows where the specified column contains NaN values
    new_df = new_df.dropna(subset=[column_name])

    return new_df



def cleanup_and_rename(df, header=1):

    df = df.copy()
    # Drop the specified number of rows from the DataFrame
    df.drop(df.index[range(header)], inplace=True)

    print(f'Shape Before: {df.shape}')

    df = clean_qty(df)
    df = filter_regions(df)
    df = clean_region_names(df)
    df = convert_to_numeric(df)
    df = manage_dates(df)
    df = drop_non_numeric_rows(df, 'PID')  #remove non numeric `Article Numbers`

    df = df.fillna(0)

    if 'Region_ID' in df.columns and 'MM_YY' in df.columns:
        df.drop(columns= ['Region_ID', 'MM_YY'], inplace = True)

    df.reset_index(drop = True, inplace = True)
    print(f'Shape After: {df.shape}','\n')
    display(df.head())
    return df

f: Get User Input (Integer)

In [ ]:
def get_user_input_int(lower: int, upper: int):
  '''User input to select the product'''

  while True:
      try:
          val = int(input('\nEnter an integer value: '))
          if val < lower or val > upper:
              raise ValueError #this will send it to the print message and back to the input option
          break
      except ValueError:
          print(f"INVALID: The number must be between ({lower} - {upper}).")
  return val


f: Add Q, Year, m_in_Q

In [ ]:
'''
def add_q_year_month(df):

  """
  Adds Quarter (Q), Year, and month within the quarter (m_in_Q) columns to the DataFrame.

  Args:
      df (DataFrame): Input DataFrame with a 'MM_YY' column.

  Returns:
      DataFrame: Modified DataFrame with Quarter, Year, and m_in_Q columns.

  Notes:
      - Deletes rows without valid 'MM_YY' data.
      - 'MM_YY' column format: 'MM_YY' (e.g., '01_21' for January 2021).
      - Q column: Month value divided by 3, rounded up.
      - Year column: Last four characters of 'MM_YY' converted to integer.
      - m_in_Q column: Month value minus Q multiplied by 3 plus 3.

  Example:
      df = pd.DataFrame({'MM_YY': ['01_21', '04_21', '07_21', '10_21', '01_22']})
      modified_df = add_q_year_month(df)
      print(modified_df)

      Output:
          MM_YY  Q  Year  m_in_Q
      0  01_21  1  2021       1
      1  04_21  2  2021       2
      2  07_21  3  2021       3
      3  10_21  4  2021       1
      4  01_22  1  2022       1
  """

  #Delete rows without valid 'MM_YY' data
  df = df[df['MM_YY'].str.get(0).isin(['0'])==True]

  #Define Quarter column
  df['Q'] = (df['MM_YY'].str[1:3].astype(float)/3).apply(np.ceil).astype(int)
  #Define Year column
  df['Year'] = df['MM_YY'].str[-4:].astype(int)
  df['m_in_Q'] = (df['MM_YY'].str[1:3].astype(int))-df['Q']*3 + 3

  #sort by Year, Q, m_in_Q
  df = df.sort_values(by=['Year', 'Q', 'm_in_Q'])

  return df
  '''

f: QTD, PreQTD, LYQTD Encoding

In [ ]:
'''
def qtd_encoder(df: pd.DataFrame):
  """
  This function add QTD, PreQTD, LYQTD Encoding for the sales report dataframe
  Three filters are created to identify rows that match specific conditions:
    - qtd_filter: selects rows from the current quarter and year.
    - preqtd_filter: selects rows from the previous quarter and year, up to and including the current month within the quarter.
    - lyqtd_filter: selects rows from the same quarter in the previous year, up to and including the current month within the quarter.
  """

  df_encoded = df.copy()

  #Create a new colum to classify QTD
  df_encoded['QTD_Encoding'] = 'O'

  #get the current Year, Quarter, and month in Quarter
  current_q = df['Q'].iloc[-1]
  current_yr = df['Year'].iloc[-1]
  current_m_in_Q = df['m_in_Q'].iloc[-1]
  print(f'Current qaurter is:\nQ = {current_q}\nYear = {current_yr}\nm_in_Q = {current_m_in_Q}\n')

  #create a filter for current quarter-to-date (QTD)
  qtd_filter = (df['Q'] == current_q) & (df['Year'] == current_yr)

  ################
  #print(qtd_filter[:10])

  #create a filter for previous quarter-to-date (PreQTD)
  preqtd_filter = (df['Q'] == current_q - 1) & (df['Year'] == current_yr) & (df['m_in_Q'] <= current_m_in_Q)

  if current_q == 1:
    preqtd_filter = (df['Q'] == 4) & (df['Year'] == (current_yr -1)) & (df['m_in_Q'] <= current_m_in_Q)


  ################
  #print(preqtd_filter[:10])

  #create a filter for last year quarter-to-date (LYQTD)
  lyqtd_filter = (df['Q'] == current_q) & (df['Year'] == (current_yr-1)) & (df['m_in_Q'] <= (current_m_in_Q))

  ################
  #print(lyqtd_filter[:10])

  df_encoded.loc[qtd_filter, 'QTD_Encoding'] = 'QTD'
  df_encoded.loc[preqtd_filter, 'QTD_Encoding'] = 'PreQTD'
  df_encoded.loc[lyqtd_filter, 'QTD_Encoding'] = 'LYQTD'


  return df_encoded
  '''

In [ ]:
'''
def ytd_encoder(df: pd.DataFrame):
    """
    Adds LYTD and YTD encoding for the sales report DataFrame.

    Args:
        df (DataFrame): Input DataFrame with columns 'Q', 'Year', and 'm_in_Q'.

    Returns:
        DataFrame: Modified DataFrame with additional 'YTD_Encoding' column.

    Notes:
        - The 'YTD_Encoding' column categorizes each row as 'YTD' or 'LYTD' based on specific conditions.
        - Two filters are created to identify rows that match specific conditions:
            - ytd_filter: Selects rows from the current year up to and including the current month.
            - lytd_filter: Selects rows from the previous year up to and including the current month.
        - The function assumes that the input DataFrame contains columns 'Q' (quarter), 'Year', and 'm_in_Q' (month within the quarter).

    """

    df_encoded = df.copy()

    # Create a new column to classify YTD
    df_encoded['YTD_Encoding'] = 'O'

    # Get the current Year, Quarter, and month in Quarter
    current_q = df['Q'].iloc[-1]
    current_yr = df['Year'].iloc[-1]
    current_m_in_Q = df['m_in_Q'].iloc[-1]
    print(f'Current quarter is:\nQ = {current_q}\nYear = {current_yr}\nm_in_Q = {current_m_in_Q}\n')

    # Create a filter for current year-to-date (YTD)
    ytd_filter = (df['Year'] == current_yr) & (df['m_in_Q'] <= current_m_in_Q) & (df['Q'] <= current_q)

    # Create a filter for last year-to-date (LYTD)
    lytd_filter = (df['Year'] == current_yr - 1) & (df['m_in_Q'] <= current_m_in_Q) & (df['Q'] <= current_q)

    df_encoded.loc[ytd_filter, 'YTD_Encoding'] = 'YTD'
    df_encoded.loc[lytd_filter, 'YTD_Encoding'] = 'LYTD'

    return df_encoded
'''

In [ ]:
#### NEW ENCODER ####
def encode_ytd(df, max_date):
    # Create 'YTD' column, label as 1 if the date belongs to the current year, else 0
    df['YTD'] = df['Date'].apply(lambda x: 1 if x.year == max_date.year and x <= max_date else 0)
    return df

def encode_lytd(df, lytd_date):
    # Create 'LYTD' column, label as 1 if the date belongs to the last year and up to the current day, else 0
    df['LYTD'] = df['Date'].apply(lambda x: 1 if x.year == lytd_date.year and x <= lytd_date else 0)
    return df

def encode_qtd(df, max_date):
    # Create 'QTD' column, label as 1 if the date belongs to the current quarter, else 0
    df['QTD'] = df['Date'].apply(lambda x: 1 if (x.year == max_date.year and x.quarter == max_date.quarter) else 0)
    return df

def encode_lyqtd(df, lyqtd_date):
    # Create 'LYQTD' column, label as 1 if the date belongs to the same quarter of the previous year and up to the current day, else 0
    df['LYQTD'] = df['Date'].apply(lambda x: 1 if x.year == lyqtd_date.year and x.quarter == lyqtd_date.quarter and x <= lyqtd_date else 0)
    return df

# Function to calculate PreQTD
def encode_lqtd(df, max_date):
    # Calculate lqtd_date and lqtd_start_date as before

    # Calculate Quarter ('Q'), Year ('Y'), and Month in Quarter ('mQ') from 'Date' column
    df['Q'] = df['Date'].dt.quarter
    df['Y'] = df['Date'].dt.year
    df['mQ'] = df['Date'].dt.month - (df['Q'] - 1) * 3

    # Determine the current quarter, year, and months in the current quarter
    current_quarter = (max_date.month - 1) // 3 + 1
    current_year = max_date.year
    current_month_in_quarter = max_date.month - (current_quarter - 1) * 3

    # Define a filter for the previous quarter-to-date (PreQTD)
    preqtd_filter = (
        (df['Q'] == current_quarter - 1) &
        (df['Y'] == current_year) &
        (df['mQ'] <= current_month_in_quarter)
    )

    if current_quarter == 1:
        preqtd_filter = (
            (df['Q'] == 4) &
            (df['Y'] == (current_year - 1)) &
            (df['mQ'] <= current_month_in_quarter)
        )

    # Create 'PreQTD' column and flag months that belong to PreQTD with 1
    df['PreQTD'] = 0
    df.loc[preqtd_filter, 'PreQTD'] = 1

    # Drop the temporary columns
    df.drop(['Q', 'Y', 'mQ'], axis=1, inplace=True)

    return df

import calendar
def encode_year_quarter(df):
    # Determine Max Date Value
    max_date = df['Date'].max()

    # Get the same day of the year in the previous year and the corresponding day of the quarter in the previous year
    # Adjust the day to 28 if the month is February and the day is 29 in a non-leap year
    if max_date.month == 2 and max_date.day == 29 and not calendar.isleap(max_date.year - 1):
        lytd_date = lyqtd_date = pd.Timestamp(year=max_date.year-1, month=max_date.month, day=28)
    else:
        lytd_date = lyqtd_date = pd.Timestamp(year=max_date.year-1, month=max_date.month, day=max_date.day)

    df = encode_ytd(df, max_date)
    df = encode_lytd(df, lytd_date)
    df = encode_qtd(df, max_date)
    df = encode_lyqtd(df, lyqtd_date)
    df = encode_lqtd(df, max_date)

    return df

In [ ]:
def add_encoding_columns(df):
    # Create 'QTD_Encoding' column {'QTD', 'LYQTD', 'PreQTD', 'O}
    df['QTD_Encoding'] = df.apply(lambda row: 'QTD' if row['QTD'] == 1 else ('LYQTD' if row['LYQTD'] == 1 else ('PreQTD' if row['PreQTD'] == 1 else 'O')), axis=1)

    # Create 'YTD_Encoding' column {'YTD', 'LYTD', 'O'}
    df['YTD_Encoding'] = df.apply(lambda row: 'YTD' if row['YTD'] == 1 else ('LYTD' if row['LYTD'] == 1 else 'O'), axis=1)

    return df

###Calculations Functions

f: Calcualte MAT

In [ ]:
from dateutil.relativedelta import relativedelta

def calculate_mat(data_frame: pd.DataFrame, product_name:str):
  """This function gets a sales dataframe that includes QTD Encoding and calcuates MAT df """
  df = data_frame.copy()
  # Ensure Date is in datetime format
  df['Date'] = pd.to_datetime(df['Date'])

  # Sort values by Date
  df = df.sort_values(by='Date')

  # Create a 'YearMonth' column for aggregation
  df['YearMonth'] = df['Date'].dt.to_period('M')

  # Group by 'YearMonth' and sum 'Sales'
  df_grouped = df.groupby(['YearMonth', 'Date', 'QTD_Encoding', 'YTD_Encoding'])[['Sales', 'Billing']].sum().reset_index()

  # Compute the 12 month moving average
  df_grouped['Sales_MAT'] = df_grouped['Sales'].rolling(window=12).sum()
  df_grouped['Billing_MAT'] = df_grouped['Billing'].rolling(window=12).sum()

  # Adding a 'Period' column with "Start Date - End Date" for 12 month period
  df_grouped['Period'] = (df_grouped['YearMonth'].dt.to_timestamp() \
                          - pd.DateOffset(months=11)).dt.to_period('M').astype(str)\
                          +  " - " + df_grouped['YearMonth'].astype(str)
  df_grouped['Product'] = product_name

  # Calculate the minimum and maximum dates in the dataset
  min_date = df['Date'].min() + relativedelta(months=10)
  max_date = df['Date'].max()

  # Apply mask after calculating MAT to limit the results to the desired date range
  mask = (df_grouped['Date'] >= min_date) & (df_grouped['Date'] <= max_date)
  df_final = df_grouped[mask]

  # Return only 'Period' and 'MAT' columns
  df_final = df_final[['Product', 'Date', 'Period', 'Sales', 'Billing', 'Billing_MAT', 'Sales_MAT', 'QTD_Encoding', 'YTD_Encoding']]

  return df_final

  '''
  #Aggregate Sales & Billing values
  group_data = df.groupby(['MM_YY', 'Q','Year', 'm_in_Q', 'QTD_Encoding', 'YTD_Encoding'],as_index=False)[['Sales', 'Billing']].sum()

  #sory by Year, Q, month in Q
  group_data = group_data.sort_values(by=['Year', 'Q', 'm_in_Q']).reset_index()
  group_data['Billing_MAT'] = group_data.Billing.rolling(window=12).sum()
  group_data['Sales_MAT'] = group_data.Sales.rolling(window=12).sum()

  # 12 month period for MAT
  group_data['Start_Period'] = group_data.MM_YY.shift(periods=11)

  # Create period labels
  group_data['Period'] = group_data['Start_Period'].str[1:] + ' - ' + group_data['MM_YY'].str[1:]

  #df_MAT['Period'].astype(str)
  group_data['Billing_MAT'].astype(float)
  group_data['Sales_MAT'].astype(float)
  group_data['YYYYMM'] = group_data['MM_YY'].astype(str).str[4:] + group_data['MM_YY'].astype(str).str[1:3]
  group_data['Date'] = pd.to_datetime(group_data['YYYYMM'], format='%Y%m', errors='coerce') + MonthEnd(1)
  group_data['Product'] = product_name

  #Subset the dataframe
  df_MAT = group_data[['Product', 'Date', 'Sales', 'Billing', 'Period', 'Billing_MAT', 'Sales_MAT', 'QTD_Encoding', 'YTD_Encoding']]




  return df_MAT
  '''

In [ ]:
def calculate_mat_group(df: pd.DataFrame, mat_by = 'MAT_by'):

  ### TEST ####
  #df = filtered_df
  #######################

  MAT_by = ['PRODUCT_GROUP', 'Region', 'Panregion']

  df_MAT = calculate_mat(df, 'CD')

  all_GROUP = df[mat_by].unique()
  #Aggregate Sales & Billing values

  for group in all_GROUP:
    if group != 0:
      df_GROUP = df[df[mat_by] == group]
      df_MAT_GROUP = calculate_mat(df_GROUP, group)
      df_MAT = pd.concat([df_MAT, df_MAT_GROUP], axis=0)

  return df_MAT

f: Caclulate Quarter Sales Trend

In [ ]:
def generate_q_sales_report(df: pd.DataFrame, product_name:str):
  ''' Calculate Quarte Sales Trend report '''

  filter_qtd_sales = (df['QTD'] == 1)
  filter_preqtd_sales = (df['PreQTD'] == 1)
  filter_lyqtd_sales = (df['LYQTD'] == 1)

  filter_ytd_sales = (df['YTD'] == 1)
  filter_lytd_sales = (df['LYTD'] == 1)


  #Sum of Sales for Quarter to Date (QTD)
  QTD = (df.loc[filter_qtd_sales].Sales.sum()).astype(float)

  #Sum of Sales for Last Year Quarter to Date (LYQTD)
  PreQTD = (df.loc[filter_preqtd_sales].Sales.sum()).astype(float)

  #Sum of Sales for Last Year Quarter to Date (LYQTD)
  LYQTD = (df.loc[filter_lyqtd_sales].Sales.sum()).astype(float)

  YTD = (df.loc[filter_ytd_sales].Sales.sum()).astype(float)
  LYTD = (df.loc[filter_lytd_sales].Sales.sum()).astype(float)

  # Data of lists.
  q_report_data = {'Product':[product_name],
          'QTD(kEUR)':[QTD/1000],  #convert to kEUR
          'PreQTD(kEUR)':[PreQTD/1000], #convert to kEUR
          'LYQTD(kEUR)':[LYQTD/1000], #convert to kEUR
          'YTD(kEUR)':[YTD/1000], #convert to kEUR
          'LYTD(kEUR)':[LYTD/1000], #convert to kEUR
          'Rel_PreQTD(%)':[(100*QTD/PreQTD).astype(int)], #convert to %
          'Rel_LYQTD(%)':[(100*QTD/LYQTD).astype(int)], #convert to %
          'Rel_LYTD(%)':[(100*YTD/LYTD).astype(int)] #convert to %
                   }

  # Create Q Report DataFrame
  q_report_df = pd.DataFrame(q_report_data)
  q_report_df = q_report_df.round({'QTD(kEUR)':1, 'PreQTD(kEUR)':1, 'LYQTD(kEUR)':1, 'Rel_PreQTD(%)':0,'Rel_LYQTD(%)':0} )

  return q_report_df

f: Calculate MAT Growth

In [ ]:
def calculate_growth(df):
    '''
    This finctions get a MAT df with QTD_Encoding and returns MAT Growth values
    - 'Base MAT'
    - 'Current MAT'
    - 'MAT Growth - Current (%)'
    - 'MAT Growth - Last Report (%)'
    - 'MAT Growth - PrevQTD (%)'
    '''

    # Drop YTD_Encoding
    df = df.drop(['YTD_Encoding'], axis=1)

    # Create an empty DataFrame to store the growth calculations
    growth_mat = pd.DataFrame()

    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Drop NaN values for the current column
        column_data = df[column].dropna()
        # Process row: Index 0
        row_0 = column_data.iloc[0]
        row_0 = pd.to_numeric(row_0, errors='coerce')

        # Process row 3
        row_3 = column_data.loc[df['QTD_Encoding'] == 'PreQTD'].iloc[-1]
        row_3 = pd.to_numeric(row_3, errors='coerce')

        # Process row 2
        row_2 = column_data.iloc[-2]
        row_2 = pd.to_numeric(row_2, errors='coerce')

        # Process row 1
        row_1 = column_data.iloc[-1]
        row_1 = pd.to_numeric(row_1, errors='coerce')

        # Calculate current MAT growth
        growth_current_mat = 100 * (row_1 - row_0) / row_0

        # Calculate last report MAT growth
        growth_last_repo_mat = 100 * (row_2 - row_0) / row_0

        # Calculate previous QTD MAT growth
        growth_preqtd_mat = 100 * (row_3 - row_0) / row_0

        # Create a Series for the column's growth calculations
        growth_column = pd.Series(
            {
                'Base MAT': column_data.index[0],
                'Current MAT': column_data.index[-1],
                'MAT Growth - Current (%)': growth_current_mat,
                'MAT Growth - Last Report (%)': growth_last_repo_mat,
                'MAT Growth - PrevQTD (%)': growth_preqtd_mat,
            }
        ).rename(column)

        # Add the column's growth calculations to the overall DataFrame
        growth_mat = pd.concat([growth_mat, growth_column], axis=1)

    # Drop the 'QTD_Encoding' column
    growth_mat = growth_mat.drop('QTD_Encoding', axis=1)

    return growth_mat

f: Calculate QTD sales by License

In [ ]:
def generate_qtd_sales_by_license(df:pd.DataFrame, product_name: str):

  #limit the data for QTD sales
  by_license_filter = df['QTD'] == 1
  df_qtd = df.loc[by_license_filter]

  #Filters
  filter_permanent = df['EID_CODE'].str.contains('PERM')
  filter_subscription = df['EID_CODE'].str.contains('SUB')
  filter_ccp = df['EID_CODE'].str.contains('CCP')
  filter_support = df['EID_CODE'].str.contains('SUPPORT')
  filter_prorated = df['EID_CODE'].str.contains('PRO-RATED')
  filter_updates = df['EID_CODE'].str.contains('UPDATE')
  filter_other = df['EID_CODE'].str.contains('OTHER')

  #Apply filters
  permanent_sales = df_qtd.Sales[filter_permanent].sum()
  permanent_qty = df_qtd.Qty[filter_permanent].sum()

  subscription_sales = df_qtd.Sales[filter_subscription].sum() + df_qtd.Sales[filter_prorated].sum()
  subscription_qty = df_qtd.Qty[filter_subscription].sum() + df_qtd.Qty[filter_prorated].sum()

  ccp_sales = (df_qtd.Sales[filter_ccp].sum() + df_qtd.Sales[filter_support].sum() + df_qtd.Sales[filter_updates].sum())
  qty_ccp = df_qtd.Qty[filter_ccp].sum() + (df_qtd.Qty[filter_support].sum() + df_qtd.Qty[filter_updates].sum())/2

  other_sales = df_qtd.Sales[filter_other].sum()
  other_qty = df_qtd.Qty[filter_other].sum()


  #  data of lists; convert to kEUR
  by_license_data = {'Product':[product_name, product_name],
          'Unit':['Sales (kEUR)', 'Seat (Qty)'],
          'Perpetual':[0.001*permanent_sales, permanent_qty],
          'Subscription':[0.001*subscription_sales, subscription_qty],
          'CCP':[0.001*ccp_sales, qty_ccp],
          'Other':[0.001*other_sales, other_qty]}

  by_license_df = pd.DataFrame(by_license_data)

  return by_license_df.round(1)


f: Add missing regions

In [ ]:
def add_missing_regions(df):

  #create an empty DataFrame
  df_all_regions = pd.DataFrame(data=None, columns = df.columns)

  #add all region with 0 sales value to make sure all regions are shown in the plot
  for region in ALL_REGIONS:
    if ~df['Region'].str.contains(region).any():
      region_dict = {'Region': region, 'MM_YY': '00', 'Sales': 0, 'EID_CODE': 'PERM', 'QTD_Encoding': 'QTD'}
      temporary_df = pd.DataFrame.from_dict([region_dict])
      df_all_regions = pd.concat([df_all_regions, temporary_df], ignore_index=True)

  #concat the df_region and df_all_regions
  df_region_concat = pd.concat([df, df_all_regions], ignore_index=True)

  return df_region_concat

f: Add missing periods to MAT

In [ ]:
def add_missing_month_MAT(df_new, df_ref):
  ######### TEST ########
  '''
  df_ref = temp_data_0
  df_new = df_MAT_new
  '''
  ########################

  df_merged = pd.merge(df_ref, df_new, suffixes=('_ref', '_new'), on='Date')

  # Create an empty DataFrame
  df_all_new = pd.DataFrame(data=None, columns=df_ref.columns)

  df_all_new['Date'] = df_merged['Date']
  df_all_new['Sales'] = df_merged['Sales_new']
  df_all_new['Billing'] = df_merged['Billing_new']
  df_all_new['Period'] = df_merged['Period_ref']
  df_all_new['Billing_MAT'] = df_merged['Billing_MAT_new']
  df_all_new['Sales_MAT'] = df_merged['Sales_MAT_new']

  # Check if 'Product_new' column is not empty before accessing the last value
  if not df_merged['Product_new'].empty:
      df_all_new.loc[:, 'Product'] = df_merged['Product_new'].iloc[-1]

  # print(df_all_new.tail(3))

  return df_all_new

f: Calculate Sales by Region & Quarter

In [ ]:
def add_panregion_column(df):
    df['Panregion'] = df['Region'].str.strip().map(ALL_PANREGIONS).fillna('NaN')
    return df

In [ ]:
def calculate_sales_by_region_and_quarter(df: pd.DataFrame, product_name: str, encoding: str):

    valid_encodings = ['QTD_Encoding', 'YTD_Encoding']

    if encoding not in valid_encodings:
        raise ValueError("Invalid encoding. Please choose from ['QTD_Encoding', 'YTD_Encoding'].")

    # Filter based on the specified encoding
    df = df.loc[df[encoding] != 'O']

    # Aggregate based on the specified encoding
    df_region_agg = df.groupby(['Region', 'Panregion', encoding])['Sales'].sum().reset_index()

    # Sort by Sales
    df_region_agg = df_region_agg.sort_values(['Region', encoding, 'Sales'], ascending=True)
    df_region_agg = df_region_agg[df_region_agg["Region"] != 0]

    df_region_agg.insert(0, 'Product', product_name)

    return df_region_agg

f: Calculate Sales by Region & Lic

In [ ]:
def calculate_sales_by_region_and_lic(df:pd.DataFrame, product_name: str):

  ### TEST #####
  #df = filtered_df

  ####

  #filter to the QTD
  df = df.loc[df['QTD'] == 1]

  #add missing regions
  df_all_regions = add_missing_regions(df)

  #Aggregate based on 'EID_CODE'
  df_region_agg = df_all_regions.groupby(['Region','Panregion', 'EID_GROUP'])['Sales'].agg('sum').reset_index()

  #sort by Sales
  df_region_agg = df_region_agg.sort_values(['Region', 'EID_GROUP', 'Sales'], ascending= True)
  df_region_agg.drop(df_region_agg[df_region_agg['Region'] == 0].index, inplace = True)

  #drop MISC & OTHER license types
  df_region_agg = df_region_agg[~df_region_agg['EID_GROUP'].str.contains('Miscsallanous')]

  df_region_agg.insert(0,'Product', product_name)

  return df_region_agg

In [ ]:
def calculate_sales_trend_q_y_region_panregion_pivot(sales_ytd, sales_qtd, col_index):
    # Create pivot table for YTD_Encoding
    pivot_ytd = sales_ytd.pivot_table(values='Sales', index=col_index, columns='YTD_Encoding', aggfunc='sum')

    # Create pivot table for QTD_Encoding
    pivot_qtd = sales_qtd.pivot_table(values='Sales', index=col_index, columns='QTD_Encoding', aggfunc='sum')

    # Merge the two pivot tables on the specified column index
    merged_pivot = pd.merge(pivot_qtd, pivot_ytd, on=col_index)

    #conver to kEUR
    merged_pivot = (merged_pivot / 1000).fillna(0)

    # Calculate the requested relative values
    merged_pivot['Rel_PreQTD(%)'] = (100 * merged_pivot['QTD'] / merged_pivot['PreQTD']).fillna(0)
    merged_pivot['Rel_LYQTD(%)'] = (100 * merged_pivot['QTD'] / merged_pivot['LYQTD']).fillna(0)
    merged_pivot['Rel_LYTD(%)'] = (100 * merged_pivot['YTD'] / merged_pivot['LYTD']).fillna(0)


    columns = ['QTD', 'PreQTD', 'LYQTD', 'Rel_PreQTD(%)', 'Rel_LYQTD(%)', 'YTD', 'LYTD', 'Rel_LYTD(%)']
    return merged_pivot[columns].round(1)

### Plot Functions

f: Plot QTD Sales by Region

In [ ]:
from matplotlib.ticker import FuncFormatter

def plot_qtd_sales_region(df: pd.DataFrame, dir: str, product_name: str, region, display=True, pallete='tab20', show_labels=True):
    #### TEST

    #df = temp_data_1
    #display = True
    #pallete = 'tab20'

    dir = DIRECTORY
    product_name = PRODUCT_GROUP

    ###### Exceptions ######
    valid_regions = ['Region', 'Panregion']

    if region not in valid_regions:
        raise ValueError("Invalid encoding. Please choose from ['Regions', 'Panregions'].")
    #######################

    # add missing regions
    df = add_missing_regions(df)

    plot_df = df.loc[df['QTD_Encoding'] == 'QTD'].copy()

    # Aggregate sum of sales by region
    plot_df = plot_df.groupby(region).agg({'Sales':'sum'}).reset_index()

    plot_df.sort_values('Sales', ascending=False, inplace=True)

    # Calculate total sales
    total_sales = plot_df['Sales'].sum()

    ### SETTINGS

    # Turn interactive plotting off
    if not display:
        plt.ioff()

    # Plot & font size
    plt.figure(figsize=[11, 5])
    plt.rcParams["font.size"] = FONT_SIZE

    #### DATA ###
    # Get Sales row of the dataframe

    # Plot values
    plot_labels = list(plot_df[region])
    plot_sizes = list(plot_df['Sales'])
    bar_plot_labels = [f'{s / total_sales * 100:0.1f}%' if show_labels else '' for l, s in zip(plot_labels, plot_sizes)]

    #### PLOT ####
    # Title
    plot_title_by_region = str(product_name) + ' - QTD Sales by ' + region + ' (' + PROCESS_MONTH + ')'

    # Labels
    plt.title(plot_title_by_region)

    numOfItemx = len(bar_plot_labels)

    # Color
    color = []
    for i in range(numOfItemx):
        color.append(gradient_color_picker(i, numOfItemx, pallete))

    # Bar plot definition
    bars = plt.bar(plot_labels, plot_sizes, color=color, edgecolor='black', linewidth=0.4)
    plt.xticks(rotation=45)

    # Adding percentages on top of the bars if show_labels is True
    for i, bar in enumerate(bars):
        if show_labels:
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), bar_plot_labels[i], ha='center', va='bottom', fontsize=FONT_SIZE)

    # Y-axis thousand comma separator
    formatter = FuncFormatter(lambda x, _: f'{x:,.0f}')
    plt.gca().yaxis.set_major_formatter(formatter)



    # Legend
    if region == 'Panregion':
        legend_labels = list(plot_df['Panregion'])
        plt.legend(bars, legend_labels)

    # ...

    #### STORE ####
    # Save as png
    file_address = dir + plot_title_by_region
    plt.savefig(file_address, bbox_inches='tight')

    if not display:
        plt.close()

    plt.show()


f: Plot (Pie): Q Sales by License

In [ ]:
def plot_qtd_sales_license(df: pd.DataFrame, dir: str, product_name: str, display=False):
    import matplotlib.ticker as ticker

    #### TEST

    '''
    df = qtd_sales_license_seat
    dir = DIRECTORY
    display = True
    product_name = 'ddsdsdsd
    '''

    ####

    df = df.loc[df['Product']==product_name]


    # Define a function to multiply numeric values by 1000
    def multiply_by_1000(x):
        if isinstance(x, (int, float)):
            return x * 1000
        else:
            return x

    # Apply the function to all numeric values of the dataframe
    df = df.applymap(multiply_by_1000)

    # SETTINGS
    # Turn interactive plotting off
    if not display:
        plt.ioff()

    # Set color
    colors = COLOR_SET_google

    # Plot and font size
    plt.figure(figsize=[7, 5])
    plt.rcParams["font.size"] = FONT_SIZE

    # DATA
    # Get Sales row of the dataframe
    df_plot = df.drop(columns=['Product', 'Unit']).sort_index(axis=1)
    df_plot = df_plot.iloc[0]

    # Plot values
    bar_plot_labels = list(df_plot.keys())  # [Perpetual, Subscription, CCP, Other]
    bar_plot_x = list(df_plot)

    # Sort the bar plot labels according to 'EID_GROUP' values
    sort_order = ['CCP', 'Subscription', 'Perpetual', 'Other']
    bar_plot_labels, bar_plot_x = zip(*sorted(zip(bar_plot_labels, bar_plot_x), key=lambda x: sort_order.index(x[0])))

    # Calculate the total sum of the values
    total_sum = sum(bar_plot_x)

    # PLOT
    # Title
    by_license_plot_title = product_name + ' - QTD Sales by License' + ' (' + PROCESS_MONTH +  ')'
    plt.title(by_license_plot_title)

    # Bar plot definition
    bars = plt.bar(bar_plot_labels, bar_plot_x, color=colors, edgecolor='black', linewidth=0.4)


    # x-axis label
    plt.xlabel('License Type')

    # Add percentage of all values to the bars
    for bar in bars:
        height = bar.get_height()
        percentage = (height / total_sum) * 100
        plt.gca().text(bar.get_x() + bar.get_width() / 2, height, '{:.1f}%'.format(percentage),
                       ha='center', va='bottom', fontsize=FONT_SIZE)

    # y-axis label with thousand separator
    plt.ylabel('Sales (EUR)')
    formatter = ticker.StrMethodFormatter('{x:,.0f}')
    plt.gca().yaxis.set_major_formatter(formatter)

    plt.xticks(rotation=0, ha='center')
    plt.yticks(rotation=0, ha='right')
    # plt.legend(loc='upper left')

    # Set graphics
    plt.grid(linestyle='--', axis='y')

    # STORE
    # Save as png
    plt.savefig(dir + '/' + by_license_plot_title, bbox_inches='tight')

    if not display:
        plt.close()

    plt.show()


f: Plot (Bar): Sales Trend

In [ ]:
def plot_q_sales_trend (df:pd.DataFrame, dir, product_name:str, display = True):
  ######## TEST ########
  '''
  df = q_report_df
  display = True
  dir = DIRECTORY
  product_name = 'SSSTHHHHH'
  '''
  #####################

  # Turn interactive plotting off
  if not display:
    plt.ioff()


  #Transform the data
  df = df.rename(columns={'QTD(kEUR)': 'QTD', 'PreQTD(kEUR)': 'PreQTD', 'LYQTD(kEUR)': 'LYQTD'})
  df = df.drop(['Rel_PreQTD(%)', 'Rel_LYQTD(%)'], axis = 1)

  plot_df = pd.melt(df, id_vars=['Product'], value_vars=['QTD', 'PreQTD', 'LYQTD'], var_name='QTD_Encoding', value_name='Sales')
  plot_df['Sales'] = plot_df['Sales']*1000

  hue_order = ['LYQTD', 'PreQTD', 'QTD']

  #set the lenth of figure based on the number of products
  num_of_products = len(plot_df['Product'].unique())
  fig_len = 1 + num_of_products * 2

  #agg_function options
  add_plot_title_phrase = ' Q-Q'
  sns_colour = sns.set_palette(COLOR_SET_google)
  linewidth = 0.4
  saturation= 0.9


  #plot & font size
  plt.figure(figsize=[fig_len,6])
  plt.rcParams["font.size"] = FONT_SIZE

  #barchart definition
  ax = sns.barplot(x='Product', y='Sales', hue= 'QTD_Encoding',
                   saturation = saturation,  palette=sns_colour,
                   edgecolor = 'black', linewidth = linewidth,
                   hue_order = hue_order,
                   errorbar = None, data = plot_df)

  #invert x axis to show larget sales at the left
  #ax.invert_xaxis()

  #Title
  plot_title = product_name +': Q-Q Short Term Trends' + ' (' + PROCESS_MONTH +  ')'

  #set labels
  plt.ylabel("Sales (EUR)")
  plt.title(plot_title)
  plt.xticks(rotation=45, ha='right')
  plt.yticks(rotation=0, ha='right')
  plt.legend(loc='upper right')

  #set graphics
  plt.grid(linestyle='--', axis='y')

  #adding commas to thousands, matplotlib, python
  ax.get_yaxis().set_major_formatter(tick.FuncFormatter(lambda x, p: format(int(x), ',')))

  #Save & show
  plt.savefig(dir + '/' +  plot_title, bbox_inches = 'tight')

  if not display:
    plt.close()

  plt.show()

f: Plot (Bar): Sales by Region

In [ ]:
def plot_sales_by_region(df: pd.DataFrame, product_name: str, dir: str, display=True, region = 'region_select', agg_function='agg_func_select'):
    ####### TEST ###########
    #df = temp_data_1
    #product_name = PRODUCT_GROUP
    #dir = DIRECTORY
    #display = True
    #agg_function = 'QTD_Encoding'
    #########################

    #add missing regions
    df = add_missing_regions(df)

    # Turn interactive plotting off
    if not display:
        plt.ioff()

    # agg_function options
    agg_func_select = ['QTD_Encoding', 'YTD_Encoding', 'EID_GROUP']
    region_select = ['Region', 'Panregion']

    if agg_function == 'QTD_Encoding':  # when QTD is selected
        add_plot_title_phrase = ' Q-Q'
        sns_colour = sns.set_palette(COLOR_SET_google)
        linewidth = 0.4

    elif agg_function == 'YTD_Encoding':  # when YTD is selected
        add_plot_title_phrase = ' Y-Y'
        sns_colour = sns.set_palette(COLOR_SET_google)
        linewidth = 0.4

    else:  # when EID_CODE is selected
        add_plot_title_phrase = ' and License'
        sns_colour = sns.color_palette(COLOR_SET_google)
        linewidth = 0.2

        # Drop rows with 'Miscellaneous' in 'EID_GROUP' column
        df = df[df['EID_GROUP'] != 'Miscellaneous'].copy()

        # Sort the data by 'Regions' and 'EID_GROUP'
        sort_order = ['CCP', 'Subscription', 'Perpetual', 'Other']
        df['EID_GROUP'] = pd.Categorical(df['EID_GROUP'], categories=sort_order, ordered=True)
        df.sort_values(by=[region, 'EID_GROUP'], inplace=True)

    # Aggregate sum of sales by region and agg_function
    df = df.groupby([region, agg_function]).agg({'Sales':'sum'}).reset_index()

    # plot & font size
    plt.rcParams["font.size"] = FONT_SIZE

    if region == 'Panregion':
      plt.figure(figsize=[12, 6])
    else:
      plt.figure(figsize=[20, 6])

    # barchart definition
    ax = sns.barplot(x=region, y='Sales', hue=agg_function,
                     saturation=0.9, palette=sns_colour, edgecolor='black',
                     linewidth=linewidth, errorbar=None, data=df)

    # Title
    by_region_plot_title = product_name + ': Sales by ' +  region + add_plot_title_phrase + ' (' + PROCESS_MONTH +  ')'

    # set labels
    plt.ylabel("Sales (EUR)")
    plt.title(by_region_plot_title)
    plt.xticks(rotation=0, ha='center')
    plt.yticks(rotation=0, ha='right')

    # Set legend location
    plt.legend(loc='upper left')

    # set graphics
    plt.grid(linestyle='--', axis='y')

    # adding commas to thousands, matplotlib, python
    ax.get_yaxis().set_major_formatter(tick.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Save & show
    plt.savefig(dir + '/' + by_region_plot_title, bbox_inches='tight')

    if not display:
        plt.close()

    plt.show()


f: Plot (Line): MAT

In [ ]:
def plot_MAT(df:pd.DataFrame, dir:str, label = '', display=True, startIndex = 0, pallete = 'PALLETE'):
  ### TEST ###
  '''
  df = df_MAT
  display = True
  dir = DIRECTORY
  startIndex = 0
  product_name = 'XXXXXX'
  pallete = 'viridis'
  '''
  ########################

  product_name = df_MAT.iloc[0]['Product']

  # Turn interactive plotting off
  if not display:
    plt.ioff()
 #Get all the products in the df
  my_products = df['Product'].unique()
  #Use the first product (Product Group)
  plot_df = df.loc[df.Product == my_products[startIndex]]

  #plot & font size
  fig, ax = plt.subplots(figsize=[15,6])
  plt.rcParams["font.size"] = FONT_SIZE

  #plot title
  plot_title_mat = product_name +': Worldwide MAT' + label + ' (' + PROCESS_MONTH +  ')'


  #y and y axes ranges
  y_max = 1.1 * max (plot_df['Billing_MAT'].max(),plot_df['Sales_MAT'].max())
  x_max = plot_df['Billing_MAT'].count()

  #Plot grid, title, and labels
  plt.ylabel('EUR')
  plt.ylim(0,y_max)
  plt.xlim(1,x_max+1)
  plt.title(plot_title_mat)


  #Add ticks
  plt.xticks(rotation=45, ha='right')
  plt.yticks(rotation=0, ha='right')

  #Set grid
  plt.grid(linestyle='dotted', axis='y')
  plt.grid(linestyle='dotted', axis='x')

  #Add thousands comma
  current_values = plt.gca().get_yticks().tolist()
  plt.gca().set_yticks(current_values)
  plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])


  flag = False
  i = 0
  for item in my_products[startIndex:]:
    #print(item)
    plot_df = df.loc[df.Product == item]
    x = plot_df['Period'].tolist()

    if i == 0 and startIndex == 0:
      new_color = gradient_color_picker(i,len(my_products), pallete)
      label = str(item) + ': Billing_MAT'
      ax.plot(x, plot_df['Billing_MAT'], color = new_color,  label= label)

    i = i + 1
    new_color = gradient_color_picker(i,len(my_products), pallete)
    label = str(item) +': Sales_MAT'
    ax.plot(x, plot_df['Sales_MAT'], color = new_color, label= label)

    #Add legend
    legend = ax.legend(loc='upper left', fontsize='x-small')

  #### STORE ####
  #Save as png
  plt.savefig(dir + '/' + plot_title_mat, bbox_inches = 'tight')

  if not display:
    plt.close()

  plt.show()


### Reshape Functions

In [ ]:
## reshape MAT data for Excel
def reshape_excel_plot_MAT(df_MAT):
    '''
    This function reshapes the MAT WW dataframe for Excel plotting.
    '''
    column_names = df_MAT['Product'].unique()

    # Initialize the DataFrame with the first product
    df_plot_temp = df_MAT.loc[df_MAT['Product'] == column_names[0]]
    df_MAT_Excel_Plot = df_plot_temp[['Date', 'Period', 'Billing_MAT', 'QTD_Encoding', 'YTD_Encoding']].copy()
    df_MAT_Excel_Plot.rename(columns={'Billing_MAT': str(column_names[0]) + ': Billing_MAT'}, inplace=True)

    # Merge data for each product into the main DataFrame
    for item in column_names:
        df_plot_temp = df_MAT.loc[df_MAT['Product'] == item]
        df_plot_temp = df_plot_temp[['Date', 'Sales_MAT']].copy()
        df_plot_temp.rename(columns={'Sales_MAT': (str(item) + ': Sales_MAT')}, inplace=True)
        df_MAT_Excel_Plot = df_MAT_Excel_Plot.merge(df_plot_temp, on='Date')
        df_MAT_Excel_Plot.set_index('Date', inplace=True)

    # Set "Period" as the index
    df_MAT_Excel_Plot = df_MAT_Excel_Plot.loc[~df_MAT_Excel_Plot['Period'].isnull()].copy()
    df_MAT_Excel_Plot.set_index('Period', inplace=True)


    # Move 'QTD_Encoding', 'YTD_Encoding' to the end
    columns_to_move_names = ['QTD_Encoding', 'YTD_Encoding']
    # Step 2: Create a new dataframe without the columns to be moved
    new_df = df_MAT_Excel_Plot.drop(columns_to_move_names, axis=1)

    # Step 3: Create a dataframe with only the columns to be moved
    columns_to_move = df_MAT_Excel_Plot[columns_to_move_names]

    # Step 4: Concatenate the new dataframe with the columns to be moved
    concatenated_df = pd.concat([new_df, columns_to_move], axis=1)

    return concatenated_df

In [ ]:
## Reshape sales by resgion for Excel
def reshape_sales_by_region(sales_by_region_and_lic):
  '''This function gets the sales_by_region_and_lic and reshape it to make Excel plot ready'''

  region_names = sales_by_region_and_lic['Region'].unique()
  df_Excel = pd.DataFrame(index=region_names)

  #Make a copy of region indexes
  df_region_names=df_Excel.copy()

  #get the name of all products
  names = sales_by_region_and_lic['Product'].unique()

  for item in names:
    #filter to the first product
    df_temp = sales_by_region_and_lic.loc[sales_by_region_and_lic['Product'] == item]
    df_temp = df_temp.groupby(by = ['Region'])['Sales'].agg(['sum'])
    df_temp.sort_values('sum', ascending = False, inplace = True)

    #Reindex df_temp to add Regions with no sales
    df_temp = df_temp.reindex(df_region_names.index, fill_value=0)

    df_temp.rename(columns = {'sum': str(item)}, inplace = True )
    df_Excel = pd.merge(df_Excel, df_temp, left_index = True, right_index = True)

  df_Excel.sort_index(ascending = True, inplace = True)

  return df_Excel

## --- Data ---

Upload Datasheet

In [ ]:
# Upload -byarticle-byregion Excel file
uploaded = files.upload()

In [ ]:
file_upload = next(iter(uploaded)) #get the uploaded file
df_upload = pd.read_excel(io.BytesIO(uploaded[file_upload])) #cast the excel file to a DataFrame

In [ ]:
print(f'The uploaded Excel file has the size of {df_upload.shape}\n')
df_upload.head() #print first 5 rows of df

In [ ]:
def keep_columns_by_index(dataframe, start_index, end_index):
    """
    Keep columns in the specified index range and drop the others.

    Parameters:
        dataframe (pd.DataFrame): The DataFrame you want to modify.
        start_index (int): The starting column index to keep (inclusive).
        end_index (int): The ending column index to keep (inclusive).

    Returns:
        pd.DataFrame: The modified DataFrame with only the specified columns.
    """
    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Input must be a Pandas DataFrame")

    if start_index < 0 or end_index < 0:
        raise ValueError("Start and end indices must be non-negative")

    if start_index > end_index or start_index >= len(dataframe.columns) or end_index >= len(dataframe.columns):
        raise ValueError("Invalid start or end index")

    # Get the list of column names to keep
    columns_to_keep = dataframe.columns[start_index:end_index + 1]

    # Keep only the specified columns and drop the rest
    dataframe_filtered = dataframe[columns_to_keep]

    return dataframe_filtered


In [ ]:
df_drop = keep_columns_by_index(df_upload, 0, 7)
df_drop.head(5)

⚠️ **IMPORTANT:** Make sure the column names and data math; otherwise, the data should be re-exported or the **Pre-processing: Renaming & Cleaning** section should be modified.
```
- Material
- Material Name
- Country (Addr. Ship-To)
- Country (Addr. Ship-To)
- Fiscal year/period
- 4.2:Billing exFr GC
- 4.4 Billing qty
- 5.2:Sales exFr GC
```

Renaming & Cleaning

In [ ]:
#Set the Headers
column_header =['PID', 'License', 'Region_ID', 'Region', 'MM_YY', 'Billing', 'Qty', 'Sales']
df_drop.columns = column_header

#Make a copy of df_read
df = cleanup_and_rename(df_drop, 1)
#see the header
print(f'\nThe size of dataframe is {df.shape}\n')

In [ ]:
PROCESS_MONTH = df['Date'].max().strftime('%Y-%m')
print(PROCESS_MONTH)

Add missing Date, PID, Region to the dataframe

In [ ]:
select_list = ['Date', 'PID', 'Region']
# Create a new DataFrame with all possible combinations of the columns in select_list
all_combinations = pd.MultiIndex.from_product([df[col].unique() for col in select_list],
                                              names=select_list).to_frame(index=False)

# Merge the new DataFrame with the original one
new_df = pd.merge(all_combinations, df, on=select_list, how='left')


In [ ]:
df = new_df.fillna(0)
df

In [ ]:
#see the header
print(f'\nThe size of dataframe is {new_df.shape}\n')

Read product license info

In [ ]:
#url address for gs workbook
workbook_url = 'https://docs.google.com/spreadsheets/d/1Gbwg4edyMHps1QQQmm_6AiR8IDUN0A_6kEtZ0pOWXz8/edit#gid=1839828940'

#Read product categories and names inot a DataFrame
product_wb = read_gs_by_url(workbook_url)

In [ ]:
#Read dataframes from gs workbook & and clean up the dataframes
df_all_lic = workbook_to_dataframe(product_wb, 'All_Licenses')
df_product_groups = workbook_to_dataframe(product_wb, 'Product_Groups')

df_all_lic = dataframe_header(df_all_lic)
df_product_groups = dataframe_header(df_product_groups)

In [ ]:
df_all_lic.head()

##Pre-processing

Drop duplicates

In [ ]:
print(f'The size of dataframe is {df_all_lic.shape}\n')
df_all_lic = df_all_lic.drop_duplicates(subset='PID', keep="first")

Merge product & sales dfs

In [ ]:
#change PID type to int
df_all_lic['PID'] = df_all_lic['PID'].astype(int)
df['PID'] = df['PID'].astype(int)

In [ ]:
#perfrom a Left JOIN on the sales table and product table
merged_df = pd.merge(df, df_all_lic, on = 'PID', how = 'left')

In [ ]:
merged_df['License'] = merged_df['LICENSE']
merged_df = merged_df.drop('LICENSE', axis =1 )
merged_df = merged_df[merged_df.Region != 0]

In [ ]:
print(f'The size of dataframe is {merged_df.shape}\n')
merged_df.head()

In [ ]:
# Create a boolean mask for the rows with "PRO-RATED"
mask = merged_df['EID_CODE'] == "PRO-RATED"

# Use loc to update the "PRO-RATED" values from days to seats
merged_df.loc[mask, 'Qty'] = merged_df.loc[mask, 'Qty'] /365


In [ ]:
merged_df.Date.max()

Encoding

In [ ]:
#Add YTD, LYTD, QTD, LYQTD, PreQTD to df
df_encoded = encode_year_quarter(merged_df)

#Add Q Y Encoding based on QTD & YTD columns
df_encoded = add_encoding_columns(df_encoded)

In [ ]:
# Add Panregion column
df_encoded.insert(4, 'Panregion', df_encoded['Region'].str.strip().map(ALL_PANREGIONS).fillna('NaN'))

## --- Select Product ---

In [ ]:
#Define productIdx as -1 before starting (DON'T RETURN TO THIS LINE)
productIdx = -1

User input

In [ ]:
# Use the previous `ProdcyIdx`. This will help the user to know which product they should next
print('Suggested Index:', productIdx + 1, '\n')

#Get user user input
group_dict = column_to_dict(df_product_groups.columns)
productIdx = get_user_input_int(0, len(group_dict)-1)

PRODUCT_GROUP = group_dict[productIdx]
group_dct = column_to_dict(df_product_groups[PRODUCT_GROUP], output = False)

## Processing

In [ ]:
df_encoded['Date'].max().strftime('%Y-%m')

In [ ]:
#Processing message
print(f'Processing Product Group: {productIdx}')

Directory

In [ ]:
#Set working directory
DIRECTORY = '/content/QBR_REPORTS/'+ str(productIdx) + '_'+ PRODUCT_GROUP + '/'
#create directory
if os.path.exists(DIRECTORY):
    shutil.rmtree(DIRECTORY)
os.makedirs(DIRECTORY)

Total: All Product in Group

In [ ]:
#========== Filter_ALL: by Articles =========#
#filter dataframe with boolean filter
boolean_filter_pid = df_encoded["CD"].str.contains(PRODUCT_GROUP) | df_encoded["SUBGROUP"].str.contains(PRODUCT_GROUP)
filtered_df = df_encoded[boolean_filter_pid]
raw_data = filtered_df.drop(['QTD_Encoding', 'YTD_Encoding'], axis = 1).copy()
print(f'Number of "{PRODUCT_GROUP}" transactions found: {filtered_df.shape[0]}\n')

In [ ]:
raw_data.sort_values(by = 'Date', ascending= True, inplace = True)
raw_data.tail()

In [ ]:
#Write raw data to excel
with pd.ExcelWriter(DIRECTORY + '/raw_data_' + PRODUCT_GROUP + '.xlsx', engine='xlsxwriter') as writer:
  raw_data.to_excel(writer, sheet_name='raw_data')

In [ ]:
#===========  Calculate_ALL: MAT ============#
#Calcualte the MAT for PRODUCT_GROUP
df_MAT = calculate_mat(filtered_df, PRODUCT_GROUP)
temp_data_0 = df_MAT #temp data for plot

#Calcualte the MAT for GROUPs & Regions
if productIdx == 0:
  df_MAT_group = calculate_mat_group(filtered_df, 'GROUP')

#Calcualte the MAT for Regions
df_MAT_region = calculate_mat_group(filtered_df, 'Region')
df_MAT_panregion = calculate_mat_group(filtered_df, 'Panregion')

#========  Calculate_ALL: Sales Trend ========#
#Calculate q repoort df for PRODUCT_GROUP
q_report_df = generate_q_sales_report(filtered_df, PRODUCT_GROUP)


#========  Calculate ALL: QTD Sales by Product ========#
#Calculate QTD Sales by PRODUCT_GROUP
qtd_sales_license_seat = generate_qtd_sales_by_license(filtered_df, PRODUCT_GROUP)

#==========  Calculate ALL: Sales by Region =========#
#Calculate Sales by Region and Quarter
sales_by_region_and_q = calculate_sales_by_region_and_quarter(filtered_df, PRODUCT_GROUP, 'QTD_Encoding')
temp_data_1 = sales_by_region_and_q #temp data for plot

#Calculate Sales by Region and License
sales_by_region_and_lic = calculate_sales_by_region_and_lic(filtered_df, PRODUCT_GROUP)
temp_data_2 = sales_by_region_and_lic #temp data for plot

#Calculate Sales by Region and YTD
sales_by_region_and_y = calculate_sales_by_region_and_quarter(filtered_df, PRODUCT_GROUP, 'YTD_Encoding')
temp_data_3 = sales_by_region_and_y #temp data for plot


#Calculate Sales Trends for Region and Panregions for Q & Y (ALL ONLY)
#set empty then try to calcluate
sales_trend_q_y_region_all = pd.DataFrame()
sales_trend_q_y_panregion_all = pd.DataFrame()

try:
  sales_trend_q_y_region_all = calculate_sales_trend_q_y_region_panregion_pivot(sales_by_region_and_y, sales_by_region_and_q, 'Region')
except:
  pass #do nothing
try:
  sales_trend_q_y_panregion_all = calculate_sales_trend_q_y_region_panregion_pivot(sales_by_region_and_y, sales_by_region_and_q, 'Panregion')
except:
  pass #do nothing

Each Product in Group

In [ ]:
#If there are more than one product in the group
if len(group_dct.values()) > 1:

  #For product in the group, except the first one
  for item in list(group_dct.values()):
    print(item)

    #========== Filter: by Articles =========#
    #Filter df_encoded by product article keywords
    boolean_filter_pid = df_encoded["CD"].str.contains(item) | df_encoded["SUBGROUP"].str.contains(item)
    filtered_df = df_encoded[boolean_filter_pid]


    #===========  Calculate: MAT ============#
    #Calculate MAT for the new filtered df
    df_MAT_new = calculate_mat(filtered_df, item)
    df_MAT_all_periods = add_missing_month_MAT(df_MAT_new, temp_data_0)
    #Append the new MAT dataframe to the end of df_MAT
    df_MAT = pd.concat([df_MAT, df_MAT_all_periods], axis=0)


    #========  Calculate: Sales Trend ========#
    #Calculate q repoort df for the new filtered df
    q_report_df_new = generate_q_sales_report(filtered_df, item)
    #Append the new q repoort df dataframe to the q repoort df
    q_report_df = pd.concat([q_report_df, q_report_df_new], axis=0)

    #========  Calculate: QTD Sales by Product ========#
    #Calculate QTD Sales by PRODUCT_GROUP
    qtd_sales_license_seat_new = generate_qtd_sales_by_license(filtered_df, item)
    qtd_sales_license_seat = pd.concat([qtd_sales_license_seat, qtd_sales_license_seat_new], axis=0)
    #Plot QTD sales by license
    plot_qtd_sales_license(qtd_sales_license_seat_new, DIRECTORY, item,  False)


    #==========  Calculate ALL: Sales by Region & Panregion =========#
    #Calculate Sales by Region and Quarter
    sales_by_region_and_q_new = calculate_sales_by_region_and_quarter(filtered_df, item, 'QTD_Encoding')
    sales_by_region_and_q = pd.concat([sales_by_region_and_q, sales_by_region_and_q_new], axis=0)
    #Plot Sales by Region & Quarter
    plot_sales_by_region(sales_by_region_and_q_new,item, DIRECTORY, False, 'Region', 'QTD_Encoding')


    #Calculate Sales by Region and Year
    sales_by_region_and_y_new = calculate_sales_by_region_and_quarter(filtered_df, item, 'YTD_Encoding')
    sales_by_region_and_y = pd.concat([sales_by_region_and_y, sales_by_region_and_y_new], axis=0)
    #Plot Sales by Region & Quarter
    plot_sales_by_region(sales_by_region_and_q_new,item, DIRECTORY, False, 'Region', 'QTD_Encoding')


    #Calculate Sales by Region and License
    sales_by_region_and_lic_new = calculate_sales_by_region_and_lic(filtered_df, item)
    sales_by_region_and_lic = pd.concat([sales_by_region_and_lic, sales_by_region_and_lic_new], axis=0)


    #Plot Sales by Region & Quarter
    plot_sales_by_region(sales_by_region_and_lic_new, item, DIRECTORY, False, 'Region', 'EID_GROUP')

## Plots

In [ ]:
plot_MAT(df_MAT, DIRECTORY, '', True, 0, 'viridis')

In [ ]:
if productIdx == 0:
  plot_MAT(df_MAT_group, DIRECTORY, ' - Product Group', True, 0,'tab10')

print('')
plot_MAT(df_MAT_region, DIRECTORY, ' - Regions' ,True, 0, 'tab20')

print('')
plot_MAT(df_MAT_panregion, DIRECTORY, ' - Panregions', True, 0, 'tab20')

In [ ]:
plot_q_sales_trend(q_report_df, DIRECTORY, PRODUCT_GROUP, True)

In [ ]:
plot_sales_by_region(temp_data_1, PRODUCT_GROUP, DIRECTORY, True, 'Region', 'QTD_Encoding')

In [ ]:
plot_sales_by_region(temp_data_1, PRODUCT_GROUP, DIRECTORY, True, 'Panregion', 'QTD_Encoding')

In [ ]:
plot_qtd_sales_license(qtd_sales_license_seat, DIRECTORY, PRODUCT_GROUP,  True)

In [ ]:
plot_sales_by_region(temp_data_2, PRODUCT_GROUP, DIRECTORY, True, 'Region', 'EID_GROUP')

In [ ]:
plot_sales_by_region(temp_data_2, PRODUCT_GROUP, DIRECTORY, True, 'Panregion', 'EID_GROUP')

In [ ]:
plot_sales_by_region(temp_data_3, PRODUCT_GROUP, DIRECTORY, True, 'Region', 'YTD_Encoding')

In [ ]:
plot_sales_by_region(temp_data_3, PRODUCT_GROUP, DIRECTORY, True, 'Panregion', 'YTD_Encoding')

In [ ]:
plot_qtd_sales_region(temp_data_1, PRODUCT_GROUP, DIRECTORY,  'Region', True, 'tab20', True)

In [ ]:
try:
    plot_qtd_sales_region(temp_data_1, PRODUCT_GROUP, DIRECTORY, 'Panregion', True, 'tab20', False)
except:
    pass  # Skip to the next command

## Wite to Excel

Reshape data

In [ ]:
#reshape MAT data
excel_MAT_WW = reshape_excel_plot_MAT(df_MAT)
excel_MAT_byGroup = reshape_excel_plot_MAT(df_MAT_group)

excel_MAT_byRegion = reshape_excel_plot_MAT(df_MAT_region)
excel_MAT_byPanregion = reshape_excel_plot_MAT(df_MAT_panregion)

In [ ]:
#Calculate growth for WW and by Group
try:
    growth_excel_MAT_WW = calculate_growth(excel_MAT_WW)
    growth_excel_MAT_byGroup = calculate_growth(excel_MAT_byGroup)
except IndexError:
    growth_excel_MAT_WW = pd.DataFrame() #empty
    growth_excel_MAT_byGroup = pd.DataFrame() #empty

In [ ]:
#reshape quarterly short term sales trend
excel_q_report_df = q_report_df.set_index('Product')

In [ ]:
#reshape QTD sales&seat by licese
excel_qtd_sales_license_seat = qtd_sales_license_seat.set_index('Product')

In [ ]:
#reshape QTD sakes by license
excel_qtd_sales_license = qtd_sales_license_seat.loc[qtd_sales_license_seat['Unit'] != 'Seat (Qty)'] #drop "Seat(Qty)"
excel_qtd_sales_license = excel_qtd_sales_license.drop(columns = ['Unit']) #drop "Unit"
excel_qtd_sales_license.set_index('Product', inplace = True) #set roduct as index

#calculate percetage per product
excel_qtd_sales_license_pct = excel_qtd_sales_license.iloc[:, 0:].apply(lambda x: x/x.sum(), axis=1)

In [ ]:
#reshpa sales by region
excel_by_region_reshaped = reshape_sales_by_region(sales_by_region_and_lic)
excel_by_region_reshaped_pct = excel_by_region_reshaped.apply(lambda x: x/x.sum() if x.sum() != 0 else 0, axis=0)

Growth from excel reshapes

Write to Excel

In [ ]:
with pd.ExcelWriter(DIRECTORY + '/_' + PRODUCT_GROUP + '_Excel_Report.xlsx', engine='xlsxwriter') as writer:
  # Write each dataframe to one ksheet.
  excel_MAT_WW.to_excel(writer, sheet_name='wwMAT')
  excel_MAT_byGroup.to_excel(writer, sheet_name='groupMAT')
  excel_MAT_byRegion.to_excel(writer, sheet_name='regMAT')
  excel_MAT_byPanregion.to_excel(writer, sheet_name='panregMAT')

  growth_excel_MAT_WW.to_excel(writer, sheet_name='wwMAT_growth')
  growth_excel_MAT_byGroup.to_excel(writer, sheet_name='groupMAT_growth')

  excel_q_report_df.to_excel(writer, sheet_name='trends_Q_Y')
  excel_qtd_sales_license_seat.to_excel(writer, sheet_name='seatLicQTD')
  excel_qtd_sales_license.to_excel(writer, sheet_name='licQTD')
  excel_qtd_sales_license_pct.to_excel(writer, sheet_name='licPctQTD')
  excel_by_region_reshaped.to_excel(writer, sheet_name='regQTD')
  excel_by_region_reshaped_pct.to_excel(writer, sheet_name='regPctQTD')
  sales_by_region_and_lic.to_excel(writer, sheet_name='regLicQTD')

  sales_by_region_and_q.to_excel(writer, sheet_name='salesRegQ')
  sales_by_region_and_y.to_excel(writer, sheet_name='salesRegY')
  sales_trend_q_y_region_all.to_excel(writer, sheet_name='trendsReg_Q_Y')
  sales_trend_q_y_panregion_all.to_excel(writer, sheet_name='trendsPanreg_Q_Y')

## File Management

In [ ]:
#Zip all reports
#Run after generating repots for all
TO_ZIP = False
if TO_ZIP == True:
  !zip -r '/content/QBR_REPORTS/QBR_Reports_CD.zip' '/content/QBR_REPORTS'
  print('The zip file is successfully created!')

In [ ]:
#============== WARNING: Delete All  ================#
#-------- Deletes Reports Folder --------#
DELETE_ALL = False
if DELETE_ALL is True:
  #============== WARNING: Delete All ================#
  DELETE_ALL = input("Are you sure you want to delete all? 'y' to confirm: ")
  if DELETE_ALL.lower() == 'y':
      print('DELETED:', DIRECTORY)
      shutil.rmtree('/content/QBR_REPORTS', ignore_errors=True)
  else:
      print("Deletion cancelled.")

Report Summary

In [ ]:
#Reporting message
print(f'Reports generated for Product Group {productIdx} \n---------------------------------------')
print('All: '+ PRODUCT_GROUP)
print(''.join("{0}\n".format(x) for x in group_dct.values()))

In [ ]:
from IPython.display import Javascript

def display_notification(message):
    js_code = f"alert('{message}');"
    display(Javascript(js_code))

# Call the function to display the notification
display_notification("All Done: The code has run to completion")